In [ ]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.debug("Logger active")

In [ ]:
import pyvast

In [ ]:
vast = pyvast.VAST(app="build/release/bin/vast")

In [ ]:
await vast.try_connect()

In [ ]:
# Select all events of type zeek.conn
data = await vast.query("#type ~ /zeek.conn/")

In [ ]:
data.shape

In [ ]:
data.schema

In [ ]:
import pandas
df = data.to_pandas()

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
import ipaddress

# Accepts a 128-bit buffer holding an IPv6 address and
# returns an IPv4 or IPv6 address.
def unpack_ip(buffer):
    num = int.from_bytes(buffer, byteorder='big')
    # convert IPv4 mapped addresses back to regular IPv4
    # https://tools.ietf.org/html/rfc4291#section-2.5.5.2
    if (num >> 32) == 65535:
        num = num - (65535 << 32)
    return ipaddress.ip_address(num)

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 15]

In [ ]:
source = 'id.orig_h'
dest = 'id.resp_h'
edgelist = df.groupby([source, dest]).size().to_frame(name='weight').reset_index()
edgelist[source] = edgelist[source].apply(unpack_ip)
edgelist[dest] = edgelist[dest].apply(unpack_ip)

In [ ]:
def draw(edgelist, with_labels=False):
    G = nx.from_pandas_edgelist(edgelist, source=source, target=dest, edge_attr='weight')
    pos = nx.kamada_kawai_layout(G, weight=None)
    edges, weights = zip(*nx.get_edge_attributes(G,'weight').items())
    weights = [x / 100 for x in weights]
    nx.draw_networkx_nodes(G, pos=pos)
    nx.draw_networkx_edges(G, pos=pos, edgelist=edges, width=weights)
    if with_labels:
        nx.draw_networkx_labels(G, pos=pos)

In [ ]:
draw(edgelist)

In [ ]:
pruned = edgelist[edgelist['weight'] > 8]
draw(pruned, with_labels=True)

In [ ]:
pruned = edgelist[edgelist['weight'] > 16]
draw(pruned, with_labels=True)

In [ ]:
pruned = edgelist[edgelist['weight'] > 24]
draw(pruned, with_labels=True)